In [1]:
# TODO: 
# - write parameters into a text file to recall them later
# - create Constants class?
# - create gui:
#      * draw layout/contents/functionality on paper (do it like a pro instead of a blockhead for once)
#      * ???
#      * profit
# - brain vision recorder LSL
# - extend marker class?
# - stop experiment on keypress?
#

import random
import numpy as np

# not sure if this is useful
class Marker:
    
    ################ standard definitions ######################
    def __init__(self,identifier):
        self.value = identifier
        
    def __str__(self):
        return 'Marker ' + str(self.value)
    
    def __repr__(self):
        return 'Marker ' + str(self.value)
    
    ################# CONTENT            #######################
        
    def identify(self): return self.value

class Sequence:
    
        def __init__(self,probt,probsd,length,prenov):
            self.sequence = []
            self.expandedSeq = []
            self.markerCount = 0
            self.amount_Target = 0
            self.amount_Novelty = {}
            self.amount_Standard = 0
        
            self.trails(probt,probsd,length,prenov)
            if self.sequence == []:
                raise Exception
        
        def trails(self,Prob_T, Probs_D,Length,PreNov):

            amount_T = round(Length * Prob_T) # get amount of Targets from propability of a Target + amount of stimuli 
            amount_Ds = {} # fill dictonary with the amounts of every distractor stimuli same as above
            
            # this will be the sum of all probabilities but the standard one, so I can see if it 
            # is even possible to create a sequence
            probs_non_standard = Prob_T
            
            
            for name, prob in Probs_D.items():   
                amount = round(prob*Length)
                amount_Ds[name] = amount
                probs_non_standard += prob 

            if probs_non_standard * PreNov > 1: 
                print(self.errorMsg(probs_non_standard,PreNov))
                self.sequence = [] # TODO: raise exception here
                pass
            '''
            this happens if you for example say you want 2 standard stimuli previous to each novelty stimuli, the whole thing
            has to be 10 stimuli long and the two novelty stimuli have to appear with a probability of 0.4
            => length 10 => 8 novelty stimuli, each has to succeed 2 standard stimuli => 3 * 8 == 24 > 10!
            => that's simply not possible
            '''
            
            # save these values so we can call them outside of this function
            self.amount_Target = amount_T
            self.amount_Novelty = amount_Ds
            self.amount_Standard = Length - amount_T - sum(amount_Ds.values())

            sequence, amount_S = self.makeSeqs(PreNov,amount_T,amount_Ds,Length) 
            ''' 
            takes the amount of standard stimuli previous to novelty stimuli, the amount of target and distractor stimuli
            and the total amount of presented stimuli and returns a list of atomic sequences and the amount of standard stimuli
            which are still free to distribute
            '''

            self.sequence = self.permutations(amount_S * [['S']]  + sequence)
            '''
            we 'randomly' choose one permutation of the list of atomic sequences
            and return it
            '''


        def makeSeqs(self,pn,aT,aDs,length):

            seq_S = pn * ['S']

            seq_T = seq_S + ['T']
            seq_Ds = []

            for name, amount in aDs.items():
                seq_Ds += amount * [seq_S + [name]]

            result = seq_Ds + aT * [seq_T]
            return result, (length - (len(result) * (pn+1)))

        # returns one possible 'random' permutation of the given list
        def permutations(self,list):
            permus = random.sample(list,len(list))    
            return flatten(permus)

        # simply creates the error message when the given parameters in trails dont fit
        def errorMsg(self,probs,prenov):
            maxPreNov = round(1 / probs)
            maxProbs = float('%.2f'%(1 / prenov))
            return '\nIt is not possible to create a sequence of stimuli with the given probabilites for novelty stimuli and the given amount of buffer previous to novelty stimuli!\
                    \nEither change the buffer to {0} at max or the sum of novelty probabilites to {1} at max'.format(maxPreNov,maxProbs)
        
        
        # write oddball steps into a list, creating a sequence of actions which then can be 
        # easily executed
        # ISI = inter stimulus interval, PSI = pre stimulus interval, 
        # scaling_jitter = scaling factor for standard distribution, make it bigger!!!
        def expand(self,ISI,PSI,standard_dev,maxValue_jitter,minValue_jitter,mean_jitter):
            # I only call the expand_sequence method here, see below
            self.expandedSeq = self.expand_sequence(
                                               ISI,
                                               PSI,
                                               standard_dev,
                                               maxValue_jitter,
                                               minValue_jitter,
                                               mean_jitter)
            
        # maps te expand_single function over the whole sequence
        # since x consists of the pattern [[...],[...],...,[...]]
        # I have to flatten it into [.....]
        def expand_sequence(self,*args):
            x = map(lambda i: self.expand_single(i,*args),self.sequence)
            return flatten(list(x))

        # just concatinate the values 
        def expand_single(self,piece,ISI,PSI,sd,mav,miv,mean):
            realISI = ISI - PSI
            return ([self.marker()] +
                    [PSI]           + 
                    [self.marker()] + 
                    [piece]         +  
                    [self.marker()] + 
                    [realISI+self.calc_jitter(sd,mav,miv,mean)]
                   )

        # pulls a value from the gausian standard distribution, magnifies it with the 
        # scaling factor and then trunctuates it with the max and min values
        def calc_jitter(self,standard_dev, max_val, min_val, mean):
            return minmax(min_val,
                          max_val,
                          round(np.random.normal(mean,standard_dev)))
        
        # creates a Marker (see above, far above)
        def marker(self):
            curm = self.markerCount
            self.markerCount = curm+1
            return Marker(curm)
        
        ####### GETTERS ########
        
        def getTargetAmount(self):   return self.amount_Target
        def getStandardAmount(self): return self.amount_Standard
        def getNoveltyAmount(self):  return self.amount_Novelty        
        def getSeq(self):            return self.sequence
        def getExpandedSeq(self):    return self.expandedSeq
        def getMarkerCount(self):    return self.markerCount
        
# concatinates a list of lists
def flatten(list):
    return [e for sublist in list for e in sublist]
    # how to read it:
    # for each sublist in the given list, for each element in the sublist, put that element into a list

# rounds a value up or down if it is not in between the given min value and max value
# minmax(5,10,6) = 6; minmax(5,10,11) =  10
def minmax(minv,maxv,x):
    t1 = min(x,maxv)
    t2 = max(t1,minv)
    return t2
        
############################################################################################################
############################################################################################################

# Testing

# define parameters
target_probability = 0.1
distractor_probabilites = {
    'd1' : 0.1,
    'none': 0.1
} 
sequence_length = 20
prenovelty_standard_stimuli = 1

# create sequence 
new_sequence = Sequence(target_probability,
                        distractor_probabilites,
                        sequence_length,
                        prenovelty_standard_stimuli)
print(new_sequence.getSeq())

print('\n\n')

standard_dev = 200
mean_jitter = 0
max_jitter = 200
min_jitter = -200
isi = 1500
psi = 500


new_sequence.expand(isi,psi,standard_dev,max_jitter,min_jitter,mean_jitter)

print(new_sequence.getExpandedSeq())




#######################################################################################################
#######################################################################################################


['S', 'S', 'S', 'S', 'S', 'none', 'S', 'none', 'S', 'd1', 'S', 'S', 'd1', 'S', 'T', 'S', 'S', 'T', 'S', 'S']



[Marker 0, 500, Marker 1, 'S', Marker 2, 1200, Marker 3, 500, Marker 4, 'S', Marker 5, 1147, Marker 6, 500, Marker 7, 'S', Marker 8, 1200, Marker 9, 500, Marker 10, 'S', Marker 11, 1190, Marker 12, 500, Marker 13, 'S', Marker 14, 800, Marker 15, 500, Marker 16, 'none', Marker 17, 1195, Marker 18, 500, Marker 19, 'S', Marker 20, 929, Marker 21, 500, Marker 22, 'none', Marker 23, 800, Marker 24, 500, Marker 25, 'S', Marker 26, 807, Marker 27, 500, Marker 28, 'd1', Marker 29, 1163, Marker 30, 500, Marker 31, 'S', Marker 32, 800, Marker 33, 500, Marker 34, 'S', Marker 35, 999, Marker 36, 500, Marker 37, 'd1', Marker 38, 1019, Marker 39, 500, Marker 40, 'S', Marker 41, 964, Marker 42, 500, Marker 43, 'T', Marker 44, 1200, Marker 45, 500, Marker 46, 'S', Marker 47, 908, Marker 48, 500, Marker 49, 'S', Marker 50, 1198, Marker 51, 500, Marker 52, 'T', Marker 53, 822, Marker 54, 500, 

In [2]:
# Here I implemented the sequence execution

from playsound import playsound # you need to install this library/ not a base library
import time
import os


# I inherit Sequence for structure
# inherit simply means that Execute has all the methods Sequence has plus some extra I define here
class Execute(Sequence):
    def __init__(self,soundDict,*args): # *args == probt,probsd,length,prenov. I was just too lazy to write it out
        super().__init__(*args) # here I call the constructor for Sequence with probt, probsd, lenght, prenov
        
        self.sound = soundDict # here I save all the sounds to be used
        
    # executes the sequence of actions
    def execute(self): 
        for action in self.getExpandedSeq():
            self.execute_action(action)    

    # execute a single action
    def execute_action(self,action):
        if isinstance(action,Marker): # in case it is a Marker object do this
            print('Sent marker %d to brain visualizer via LSL' % action.identify())
        elif isinstance(action,str): # String object aka a sound
            playsound(self.sound[action])
        elif isinstance(action,(int,float)): # int or float aka wait time
            time.sleep(action/1000) # time.sleep pauses the system for the number given in seconds, so I have to divide it by 1k
            
            
            
######################################################################################################### 
#######################   Testing
os.chdir('C:/Users/suuuj/Desktop/training room')
            
sounds = {
    'T' : 'baboon.mp3',
    'D1' : 'bark.mp3',
    'S' : 'batschatter.mp3'
}

probD = {
    'D1' : 0.1
}

standard_dev = 1
mean_jitter = 0
max_jitter = 200
min_jitter = -200
scaling_jitter = 200
isi = 1500
psi = 500

x = Execute(sounds,0.1,probD,20,1) # create Execute object

print(x.getSeq())

x.expand(isi,psi,standard_dev,max_jitter,min_jitter,mean_jitter) # create actions sequence

print(x.getExpandedSeq())

x.execute() # execute axtion sequence


['S', 'S', 'T', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'D1', 'S', 'D1', 'S', 'S', 'S', 'S', 'S', 'T', 'S']
[Marker 0, 500, Marker 1, 'S', Marker 2, 1000, Marker 3, 500, Marker 4, 'S', Marker 5, 1002, Marker 6, 500, Marker 7, 'T', Marker 8, 999, Marker 9, 500, Marker 10, 'S', Marker 11, 1000, Marker 12, 500, Marker 13, 'S', Marker 14, 1000, Marker 15, 500, Marker 16, 'S', Marker 17, 1000, Marker 18, 500, Marker 19, 'S', Marker 20, 1000, Marker 21, 500, Marker 22, 'S', Marker 23, 999, Marker 24, 500, Marker 25, 'S', Marker 26, 999, Marker 27, 500, Marker 28, 'S', Marker 29, 1002, Marker 30, 500, Marker 31, 'D1', Marker 32, 999, Marker 33, 500, Marker 34, 'S', Marker 35, 998, Marker 36, 500, Marker 37, 'D1', Marker 38, 1001, Marker 39, 500, Marker 40, 'S', Marker 41, 1001, Marker 42, 500, Marker 43, 'S', Marker 44, 1001, Marker 45, 500, Marker 46, 'S', Marker 47, 999, Marker 48, 500, Marker 49, 'S', Marker 50, 999, Marker 51, 500, Marker 52, 'S', Marker 53, 1001, Marker 54, 500, Marker 55, 'T

In [ ]:
from tkinter import * # you need to install this / not in base library
from tkinter import filedialog
import Pmw
import ntpath

def button(master,text,side=LEFT,command=None):
    w = Button(master,text=text,command=command)
    w.pack(expand=YES,side=side,fill=BOTH)
    return w


    
# global Parameters
bwOuter = 5
bwInner = 3
    
class Gui(Frame):
    
    
    def __init__(self):
        Frame.__init__(self)
        self.pack(expand=YES,fill=BOTH)
        self.master.minsize(1000,200)
        
        self.initParameters()
        self.initStimGui()
        self.initParamGui()
        
    def initParameters(self):
        
        self.seq = None
        
        self.leftover_prob = 1
        self.distCount = 0
        
        self.distDict = {}
        self.audios = {}
        
        self.target_prob = 0
        
        self.targetIden = 'Target'
        self.standardIden = 'Standard'
        self.distIden = 'Distractor'
        
        # Stimulus modifcation gui parameters
        
        self.curStimLB = None
        self.curStimLB_index = END
        
        # values for sequence calculation
        
        self.isi = None
        self.psi = None
        self.pnss = None
        self.jittersd = None
        self.jittermean = None
        self.jittermax = None
        self.jittermin = None
             
        # Param Labels + Entry Parameters
        
        self.startButton = None
        self.ready = False
        self.pebal = None
        
            # entries 
            
        self.isi_entry        = None
        self.psi_entry        = None
        self.pnss_entry       = None
        self.jittersd_entry   = None
        self.jittermean_entry = None
        self.jittermax_entry  = None
        self.jittermin_entry  = None
        self.length           = None
        
        self.entries = None
        
            # labels + descriptions
            
        self.isi_label           = 'ISI'
        self.psi_label           = 'PSI'
        self.pnss_label          = 'Pre-Novelty-Stimulus Standard Stimuli'
        self.jittersd_label      = 'Jitter Standard Derivaiton'
        self.jittermean_label    = 'Jitter Mean Value'
        self.jittermax_label     = 'Jitter Max Value'
        self.jittermin_label     = 'Jitter Min Value'
        self.length_label        = 'Length'
        
        self.isi_desc        = 'Inter Stimulus Interval'
        self.psi_desc        = 'Pre Stimulus Interval'
        self.pnss_desc       = 'Amount of Standard Stimuli that need to preceed a Novelty Stimulus'
        self.jittersd_desc   = 'SD of gausian distribution used for creating the jitter'
        self.jittermean_desc = 'mean for the gausian distribution used for creating the jitter'
        self.jittermax_desc  = 'max value the jitter can have'
        self.jittermin_desc  = 'min value the jitter can have'
        self.length_desc     = 'Length of Oddball in auditory stimuli'
        
        # create Stimuli parameters
        
        self.csDialog = None
        
        self.bt_label_cs = 'Create Stimulus'
        self.bt_label_cancel = 'Cancel'
        
            # I want only one Target and one Standard Stimulus
        self.noTarget = True
        self.noStandard = True
        
        self.prob_entry = None
        self.prob_label = 'Probability'
        self.peval = 0
        
        self.openaudio_bt_label = 'Open Audio File'
        self.audio_entry = None
        
        self.filepath = ''
        
        
        
    # Gui + functionality for creating and modifying stimuli
    
    def initStimGui(self):
        
        stim_frame = Frame(self,borderwidth=bwOuter,relief=SUNKEN)
        stim_frame.pack(side=LEFT,expand=YES,fill=BOTH)
        
        # Buttons
        
        buttonBox = Pmw.ButtonBox(stim_frame,
                                  labelpos='nw',
                                  label_text='Modify Stimuli',
                                  frame_borderwidth=bwInner,
                                  frame_relief=GROOVE)
        buttonBox.pack(fill=X,padx=10,pady=10)
        
        buttonBox.add('Create Stimuli', command = self.createStim)
        buttonBox.add('Remove Stimuli', command = self.removeStim)
        buttonBox.add('Change Stimuli', command = self.changeStim)
        
        # Listbox for created Stimuli
        
        lbf = Frame(stim_frame,
                    borderwidth = bwInner)
        lbf.pack(expand=YES,fill=BOTH)
        
        self.curStimLB = Listbox(lbf)
        self.curStimLB.pack(side=LEFT,expand=YES,fill=BOTH)
        
        scrolly = Scrollbar(lbf,command=self.curStimLB.yview)
        scrolly.pack(side=RIGHT,fill=BOTH)
        
        self.curStimLB.configure(yscrollcommand = scrolly.set)
        
    def changeStim(self):
        
            self.curStimLB_index = self.curStimLB.curselection()            
            indf = self.curStimLB.get(self.curStimLB_index)
            
            
            prob, path = self.parse(indf.split())
            
            self.peval = prob
            self.filepath = path
            
            self.removeStim()
            self.createStim()
            
    def parse(self,indf):
        if indf[1] == self.distIden:
            return float(indf[4]), indf[6]
        else: 
            return float(indf[3]), indf[5]
    
    def removeStim(self):
        
        try:
            indx = self.curStimLB.curselection()
            stim_idens = self.curStimLB.get(indx).split()
            stim_type = stim_idens[1]
            self.curStimLB.delete(indx)
            
            if stim_type == self.standardIden:
                del(self.audios['S'])
                self.noStandard = True
                self.leftover_prob += float(stim_idens[4])
            elif stim_type == self.targetIden:
                del(self.audios['T'])
                self.noTarget = True
                self.leftover_prob += self.target_prob
                self.target_prob = 0
            elif stim_type == self.distIden:
                iden = 'd%s' % stim_idens[2]
                print(iden)
                self.leftover_prob += self.distDict[iden]
                del(self.distDict[iden])
                del(self.audios[iden])
        except:
            self.errorWin('Select a stimulus you want to remove!')
            
        
    def createStim(self):
        
        type_lb = None
        
        self.csDialog = Pmw.Dialog(self,
                                  buttons=(self.bt_label_cs,self.bt_label_cancel),
                                  command = lambda i: self.execute(type_lb,i))
              
        
        # stim prob
        
        prob_frame = Frame(self.csDialog.interior(),relief=SUNKEN,borderwidth=bwOuter)
        prob_frame.pack(side=LEFT,fill=Y)
        
        Label(prob_frame,text='Maximum probability : %.3f' % self.leftover_prob).pack()
        
        self.prob_entry = Pmw.EntryField(prob_frame,
                                        labelpos=N,
                                        label_text=self.prob_label,
                                        validate = {
                                            'validator' : 'real',
                                            'max' : self.leftover_prob + 0.0001
                                        })
        
        self.prob_entry.setvalue(self.peval)
        
        self.prob_entry.pack(side=LEFT,expand=1,fill=X,padx=5)
        
        # stim type
        
        
        stim_type_frame = Frame(self.csDialog.interior(),relief=SUNKEN,borderwidth=bwOuter)
        stim_type_frame.pack(side=LEFT,fill=Y)
        
        Label(stim_type_frame,text='Stimulus type',relief=FLAT,borderwidth=bwInner).pack(padx=3,pady=3)
        
        type_lb = Listbox(stim_type_frame,height=3)
        type_lb.insert(END,self.distIden)
        if self.noTarget: type_lb.insert(END,self.targetIden)
        if self.noStandard: type_lb.insert(END,self.standardIden)
            
        type_lb.pack(side=LEFT,expand=1,fill=X)
        
        # audio file
        
        audio_frame = Frame(self.csDialog.interior(),relief=SUNKEN,borderwidth=bwOuter)
        audio_frame.pack(side=LEFT,fill=BOTH,expand=1)
        
        self.audio_entry = StringVar()
        self.audio_entry.set(self.filepath)
        
        Button(audio_frame,text=self.openaudio_bt_label,command=self.filedialog).pack(padx=5,pady=5,expand=1,fill=X)
        Entry(audio_frame,state=DISABLED,textvariable=self.audio_entry).pack(padx=5,pady=5,expand=1,fill=X)
        
        # dialog.show() allows you still access to the parent, dialog.activate() does not
        
        self.csDialog.activate()
        
    def filedialog(self):
        self.audio_entry.set(filedialog.askopenfilename())
        
    def execute(self,lb,iden):
        if iden == self.bt_label_cs:
            k = 0
            try: 
                stimType = lb.get(lb.curselection())                
            except:
                self.errorWin('Specify a stimulus type!')  
                k = 1
            
            prob = float(self.prob_entry.getvalue())            
            if prob <= 0:
                self.errorWin('Thou jest? Select a probability greater than 0!')
                k = 1
            
            
            path = self.audio_entry.get()
            if path == '':
                self.errorWin('Specify a file path!')
                k = 1

                
            if k == 0:
                name = self.createEntry(stimType,prob,path)
                
                self.curStimLB.insert(self.curStimLB_index,
                                     'Alias: %s ;Probabiliy: %.3f ;Audiofile: %s' 
                                      % 
                                      (name,prob,ntpath.basename(path))
                                     )
                
                self.curStimLB_index = END
                self.peval = 0
                self.filepath = ''
                self.csDialog.destroy()
            
        elif iden == self.bt_label_cancel:
            self.peval = 0
            self.filepath = ''
            self.csDialog.destroy()
            
    def createEntry(self,t,p,fp):
        if t == self.targetIden:
            self.target_prob = p
            self.audios['T'] = fp
            self.noTarget = False
            name = self.targetIden
        elif t == self.standardIden:
            self.audios['S'] = fp
            self.noStandard = False
            name = self.standardIden
        elif t == self.distIden:
            iden = 'd%d' % self.distCount            
            name = self.distIden + ' %d' % self.distCount 
            self.distCount += 1
            self.distDict[iden] = p
            self.audios[iden] = fp
            
        self.leftover_prob -= p
        return name
        
    
    # Gui + functionality for Parameter Input + Starting the actual Oddball
    
    def initParamGui(self):
        
        param_frame = Frame(self,borderwidth=bwOuter,relief=SUNKEN)
        param_frame.pack(side=RIGHT,fill=BOTH,expand=YES)
        
        
        # Parameter Entries
        entry_frame = Frame(param_frame,relief=GROOVE,borderwidth=bwOuter)
        entry_frame.pack(expand = YES, fill=BOTH)
        
        self.pebal = Pmw.Balloon(entry_frame)
        
        self.isi_entry        = self.labelentry(entry_frame, self.isi_label,        self.isi_desc)
        self.psi_entry        = self.labelentry(entry_frame, self.psi_label,        self.psi_desc)
        self.pnss_entry       = self.labelentry(entry_frame, self.pnss_label,       self.pnss_desc)
        self.jittersd_entry   = self.labelentry(entry_frame, self.jittersd_label,   self.jittersd_desc)
        self.jittermean_entry = self.labelentry(entry_frame, self.jittermean_label, self.jittermean_desc)
        self.jittermax_entry  = self.labelentry(entry_frame, self.jittermax_label,  self.jittermax_desc)
        self.jittermin_entry  = self.labelentry(entry_frame, self.jittermin_label,  self.jittermin_desc)
        self.length_entry     = self.labelentry(entry_frame, self.length_label,     self.length_desc)
        
        self.entries = [
            self.isi_entry,
            self.psi_entry,
            self.pnss_entry,
            self.jittersd_entry,
            self.jittermean_entry,
            self.jittermax_entry,
            self.jittermin_entry,
            self.length_entry
        ]

        for entry in self.entries:
            entry.pack(fill=X, expand=YES, padx=10,pady=5)
            
        Pmw.alignlabels(self.entries)
        
        # Connect LSL + Start Exp Buttons
        
        but_frame = Frame(param_frame,borderwidth=bwInner,relief=GROOVE)
        but_frame.pack(expand=1,fill=X,padx=10,pady=10)
        
        Button(but_frame,text='Initialize LSL Outlet',command=self.initLSL).pack(side=LEFT,expand=1,fill=X)
        Button(but_frame,text='Initialize Oddball Sequence',command=self.initSeq).pack(side=LEFT,expand=1,fill=X)
        self.startButton = Button(but_frame,text='Start Sequence',state=DISABLED,command = self.startSeq)
        self.startButton.pack(side=LEFT,expand=1,fill=X)
        
    def initLSL(self):
        k = 0
        
        try:
            self.isi = float(self.entries[0].getvalue())
            self.psi = float(self.entries[1].getvalue())
            self.pnss = int(self.entries[2].getvalue())
            self.jittersd = float(self.entries[3].getvalue())
            self.jittermean = float(self.entries[4].getvalue())
            self.jittermax = float(self.entries[5].getvalue())
            self.jittermin = float(self.entries[6].getvalue())
            self.length = int(self.entries[7].getvalue())
        except:
            self.errorWin('Some parameter fields were either not or falsely specified')
            k = 1
            
        # start LSL
        if k == 0:
            if self.ready: self.startButton['state'] = NORMAL
            self.errorWin('Initialization Successful!')
            self.ready = True
            
    def initSeq(self): 
        
        if (not self.noTarget) & (not self.noStandard):
            self.seq = Execute(self.audios,
                               self.target_prob,
                               self.distDict,
                               self.length,
                               self.pnss)
            self.seq.expand(self.isi,
                            self.psi,
                            self.jittersd,
                            self.jittermax,
                            self.jittermin,
                            self.jittermean
                           )
            if self.ready: 
                self.startButton['state'] = NORMAL
            self.ready = True
        else: 
            self.errorWin('You need to specify at least a target and standard stimulus in order to create a sequence!')

    def startSeq(self): 
        self.seq.execute()
    
    # Helper functions
    
    def labelentry(self,master,text='',desc=''):
        w = Pmw.EntryField(master,labelpos = W,labelmargin=20,label_text=text)
        self.pebal.bind(w,desc)
        return w
    
    def errorWin(self,msg):
        errorDia = Pmw.Dialog(self,buttons=('Got it','Understood'),command=lambda _: errorDia.destroy())
        
        Label(errorDia.interior(),text=msg,relief=SUNKEN,borderwidth=bwInner).pack(pady=5,padx=5)
        
        errorDia.activate()
            
        
        
        
        

Gui().mainloop()
    
    
    
